In [1]:
pip install python-docx

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from docx import Document
import csv

In [3]:
def docx_table_to_csv(docx_path, csv_path):
    
    # 워드 문서 열기
    doc = Document(docx_path)
    
    # 첫 번째 표 선택
    table = doc.tables[0]

    # CSV 파일 쓰기
    with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        for row in table.rows:
            csv_row = []
            for cell in row.cells:
                csv_row.append(cell.text.strip())
            csvwriter.writerow(csv_row)

In [4]:
# 워드 파일 경로와 CSV 파일 경로 지정
docx_path = './ISO 국가코드.docx'
csv_path = 'ISO_code.csv'

In [5]:
import pandas as pd
iso_code = pd.read_csv('./ISO_code.csv')

In [6]:
name_list = list(iso_code['한글명'])

In [7]:
len(name_list)

245

In [8]:
df_iso = pd.read_csv('./ISO_code.csv')

In [9]:
df_iso

,3자리코드,한글명,영문명
0,GHA,가나,GHANA
1,GAB,가봉,GABON
2,GUY,가이아나,GUYANA
3,GMB,감비아,GAMBIA
4,GGY,건지 섬,GUERNSEY
...,...,...,...
240,PHL,필리핀,PHILIPPINES
241,PCN,핏케언 제도,PITCAIRN
242,HMD,허드 맥도널드 제도,HEARD AND MC DONALD ISLANDS
243,HUN,헝가리,HUNGARY


In [10]:
import requests
import json
import csv
import os

# API 요청과 JSON 파싱, 각 항목을 개별 CSV 파일로 저장하는 함수
def fetch_and_save_data(name_list, output_csv_path):
    url = 'http://apis.data.go.kr/1262000/CountryFlagService2/getCountryFlagList2'
    params ={'serviceKey' : 'r2RSwYUKhfJv4vhTgtWZ3pzuOsY9Oj6i80TMtWRpxJm6ksvgh5aHlZbwX+17bd0DhSqzu6j1UEAtCRYMHSJzvA==', 'returnType' : 'JSON'}

    # 모든 데이터를 수집할 리스트
    all_data = []

    for query in name_list:
        params['cond[country_nm::EQ]'] = query
        response = requests.get(url, params=params)
        response_content = response.json()  # JSON 데이터로 파싱

        # JSON 데이터에서 'data' 항목 추출
        data_list = response_content.get('data', [])

        for item in data_list:
            # 필요한 데이터 추출
            entry = {
                'content_ty': item.get('content_ty', ''),
                'country_eng_nm': item.get('country_eng_nm', ''),
                'country_iso_alp2': item.get('country_iso_alp2', ''),
                'country_nm': item.get('country_nm', ''),
                'download_url': item.get('download_url', '')
            }
            all_data.append(entry)
            
        # 수집된 모든 데이터를 하나의 CSV 파일로 저장
        with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['content_ty', 'country_eng_nm', 'country_iso_alp2', 'country_nm', 'download_url', 'origin_file_nm']
            csvwriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
            # 헤더 작성
            csvwriter.writeheader()
    
            # 데이터 작성
            for data in all_data:
                csvwriter.writerow(data)

# 전체 작업을 수행하는 메인 함수
def main():
    output_csv_path = './country_jsondata.csv'
    
    fetch_and_save_data(name_list,  output_csv_path)
    
    print(f"CSV 파일이 {output_csv_path}에 저장되었습니다.")

if __name__ == '__main__':
    main()

CSV 파일이 ./country_jsondata.csv에 저장되었습니다.


In [11]:
df = pd.read_csv('./country_jsondata.csv').drop(columns={'content_ty', 'origin_file_nm'}).dropna()

In [12]:
df = df.rename(columns={'country_eng_nm':'영문명','country_iso_alp2':'2자리코드','country_nm':'한글명','download_url':'이미지'})

In [13]:
df['영문명'] = df['영문명'].str.upper()

In [14]:
df.drop_duplicates()

,영문명,2자리코드,한글명,이미지
0,GHANA,GH,가나,https://opendata.mofa.go.kr:8444/fileDownload/...
1,GABON,GA,가봉,https://opendata.mofa.go.kr:8444/fileDownload/...
2,GUYANA,GY,가이아나,https://opendata.mofa.go.kr:8444/fileDownload/...
3,GAMBIA,GM,감비아,https://opendata.mofa.go.kr:8444/fileDownload/...
4,GUATEMALA,GT,과테말라,https://opendata.mofa.go.kr:8444/fileDownload/...
...,...,...,...,...
169,FIJI,FJ,피지,https://opendata.mofa.go.kr:8444/fileDownload/...
170,FINLAND,FI,핀란드,https://opendata.mofa.go.kr:8444/fileDownload/...
171,PHILIPPINES,PH,필리핀,https://opendata.mofa.go.kr:8444/fileDownload/...
172,HUNGARY,HU,헝가리,https://opendata.mofa.go.kr:8444/fileDownload/...


In [15]:
country_data = pd.merge(df_iso, df, on=['영문명','한글명'], how='left').dropna().reset_index(drop=True)

In [16]:
country_data

,3자리코드,한글명,영문명,2자리코드,이미지
0,GHA,가나,GHANA,GH,https://opendata.mofa.go.kr:8444/fileDownload/...
1,GAB,가봉,GABON,GA,https://opendata.mofa.go.kr:8444/fileDownload/...
2,GUY,가이아나,GUYANA,GY,https://opendata.mofa.go.kr:8444/fileDownload/...
3,GMB,감비아,GAMBIA,GM,https://opendata.mofa.go.kr:8444/fileDownload/...
4,GTM,과테말라,GUATEMALA,GT,https://opendata.mofa.go.kr:8444/fileDownload/...
...,...,...,...,...,...
147,PYF,프랑스령 폴리네시아,FRENCH POLYNESIA,PF,https://opendata.mofa.go.kr:8444/fileDownload/...
148,FJI,피지,FIJI,FJ,https://opendata.mofa.go.kr:8444/fileDownload/...
149,FIN,핀란드,FINLAND,FI,https://opendata.mofa.go.kr:8444/fileDownload/...
150,PHL,필리핀,PHILIPPINES,PH,https://opendata.mofa.go.kr:8444/fileDownload/...


In [17]:
# CSV 파일로 저장
country_data.to_csv('./country_data.csv', index=False, encoding='utf-8')